In [255]:
from AMRGraph import AMR
from AMRData import CustomizedAMR
from utilities import pretty_print, generate_action_sequence, generate_custom_amr, generate_amr_with_literals
import ActionSequenceGenerator
import TokensReplacer
import re
import itertools
from operator import itemgetter

sentence = """A staff of 30 specialists will conduct research and training on cyber warfare and will be prepared to assist NATO members with response to any future cyber attacks ."""
amr_string = """(a / and~e.8,13 
      :op1 (r / research-01~e.7 
            :ARG0 (s / staff~e.1 
                  :consist-of~e.2 (p2 / person~e.4 :quant 30~e.3 
                        :ARG0-of~e.4 (s2 / specialize-01~e.4))) 
            :ARG1~e.10 (w / warfare~e.12 
                  :mod (c / cyber~e.11))) 
      :op2 (t / train-01~e.9 
            :ARG1 w 
            :ARG2 s) 
      :op3 (p / prepare-02~e.16 
            :ARG1 s 
            :ARG2 (a2 / assist-01~e.18 
                  :ARG0 s 
                  :ARG1 (c3 / country 
                        :ARG0-of (h / have-org-role-91 
                              :ARG1 (m3 / military :wiki "NATO" 
                                    :name (n2 / name :op1 "NATO"~e.19)) 
                              :ARG2 (m / member~e.20))) 
                  :ARG2~e.21 (r2 / respond-01~e.22 
                        :ARG1~e.23 (a3 / attack-01~e.27 
                              :mod (c2 / cyber~e.26) 
                              :mod (f / future~e.25) 
                              :mod (a4 / any~e.24))))))
"""

In [256]:
amr = AMR.parse_string(amr_string)

In [257]:
custom_amr = generate_custom_amr(amr)


Mappings between node variables and their corresponding concepts.

{'p2': 'person', 'a': 'and', 'c': 'cyber', 'm3': 'military', 'r2': 'respond-01', 'f': 'future', 's2': 'specialize-01', 'm': 'member', 'a3': 'attack-01', 'p': 'prepare-02', 's': 'staff', 'r': 'research-01', 't': 'train-01', 'w': 'warfare', 'h': 'have-org-role-91', 'c3': 'country', 'c2': 'cyber', 'n2': 'name', 'a2': 'assist-01', 'a4': 'any'}

Mappings between nodes and all the aligned tokens: If the nodes don't havea variable (polarity, literals, quantities, interrogatives), they specify both the aligned tokens and the parent in order to uniquely identify them

{'p2': ['4'], 'a': ['8', '13'], 'c': ['11'], 'r2': ['22'], 'f': ['25'], 's2': ['4'], '30': [('3', 'p2')], 'm': ['20'], 's': ['1'], 'p': ['16'], 'a3': ['27'], 'r': ['7'], u'NATO': [(u'19', 'n2')], 't': ['9'], 'w': ['12'], 'c2': ['26'], 'a2': ['18'], 'a4': ['24']}

Mappings between relations and tokens. Uniquely identified by also specifying the parent of that relat

In [258]:
# get the unaligned have-org-role-91 nodes which have an ARG1 child, i.e. they can be replaced
have_org_role_nodes = [k for k in amr.node_to_concepts if amr.node_to_concepts[k] == 'have-org-role-91'
                                                          and k not in amr.node_to_tokens
                                                          and amr[k]
                                                          and "ARG1" in amr[k]]

In [259]:
have_org_role_nodes

['h']

Things to update:
- Remove have-org-role-91 from amr.node_to_concepts
- Replace the have-org-role-91 node from amr with its child having the relation "ARG1"
- Update the parent of have-org-role-91 to point to the new node

In [260]:
amr.node_to_concepts = dict((k, v) for k, v in amr.node_to_concepts.iteritems() if k not in have_org_role_nodes)

In [261]:
amr.node_to_concepts

{'a': 'and',
 'a2': 'assist-01',
 'a3': 'attack-01',
 'a4': 'any',
 'c': 'cyber',
 'c2': 'cyber',
 'c3': 'country',
 'f': 'future',
 'm': 'member',
 'm3': 'military',
 'n2': 'name',
 'p': 'prepare-02',
 'p2': 'person',
 'r': 'research-01',
 'r2': 'respond-01',
 's': 'staff',
 's2': 'specialize-01',
 't': 'train-01',
 'w': 'warfare'}

In [262]:
import copy

for h in have_org_role_nodes:
    node = amr.pop(h)
    new_node = node['ARG1']
    # add the have_org_role_children to the node corresponding to its ARG1 child
    node.pop('ARG1')
    for rel in node:
        amr[new_node[0]].append(rel, node[rel])
    # update the parent of have_org_role
    for k in amr:
        for rel in amr[k]:
            if amr[k][rel][0] == h:
                amr[k].replace(rel, new_node)

In [263]:
print amr

defaultdict(<class 'util.ListMap'>, {'s2': defaultdict(<type 'list'>, {}), u'NATO': defaultdict(<type 'list'>, {}), 'm3': defaultdict(<type 'list'>, {'wiki': [(u'NATO',)], 'ARG2': [('m',)], 'name': [('n2',)]}), 'a': defaultdict(<type 'list'>, {'op1': [('r',)], 'op2': [('t',)], 'op3': [('p',)]}), 'a3': defaultdict(<type 'list'>, {'mod': [('c2',), ('f',), ('a4',)]}), 'a2': defaultdict(<type 'list'>, {'ARG0': [('s',)], 'ARG1': [('c3',)], 'ARG2': [('r2',)]}), 'a4': defaultdict(<type 'list'>, {}), 'c3': defaultdict(<type 'list'>, {'ARG0-of': [('m3',)]}), 'c2': defaultdict(<type 'list'>, {}), 'p2': defaultdict(<type 'list'>, {'quant': [('30',)], 'ARG0-of': [('s2',)]}), 'c': defaultdict(<type 'list'>, {}), 'r2': defaultdict(<type 'list'>, {'ARG1': [('a3',)]}), 'f': defaultdict(<type 'list'>, {}), '30': defaultdict(<type 'list'>, {}), 'm': defaultdict(<type 'list'>, {}), 'p': defaultdict(<type 'list'>, {'ARG1': [('s',)], 'ARG2': [('a2',)]}), 's': defaultdict(<type 'list'>, {'consist-of': [('p2

Apply the name entity replacer on the new amr:

In [264]:
(new_amr, new_sentence, named_entities) = TokensReplacer.replace_named_entities(amr, sentence)

In [265]:
print new_sentence

A staff of 30 specialists will conduct research and training on cyber warfare and will be prepared to assist military members with response to any future cyber attacks .


In [266]:
print named_entities

[('m3', 'n2', [u'NATO'], 19, 19)]
